In [4]:
import numpy as np

class SimplePipeline:
    def __init__(self, steps):
        self.steps = steps

    def fit(self, X, y=None):
        for name, step in self.steps:
            if hasattr(step, 'fit'):
                step.fit(X, y)
            if hasattr(step, 'transform'):
                X = step.transform(X)
        self.final_model = self.steps[-1][1]
        return self

    def predict(self, X):
        for name, step in self.steps[:-1]:
            if hasattr(step, 'transform'):
                X = step.transform(X)
        return self.final_model.predict(X)

class Scaler:
    def fit(self, X, y=None):
        self.mean = np.mean(X, axis=0)
        self.std = np.std(X, axis=0)
        return self
    
    def transform(self, X):
        return (X - self.mean) / self.std

class OneHotEncoder:
    def fit(self, X, y=None):
        self.unique_values = [np.unique(X[:, i]) for i in range(X.shape[1])]
        return self
    
    def transform(self, X):
        encoded_features = []
        for i in range(X.shape[1]):
            encoded = (X[:, i][:, None] == self.unique_values[i]).astype(int)
            encoded_features.append(encoded)
        return np.hstack(encoded_features)

class MostFrequentClassifier:
    def fit(self, X, y):
        unique_classes, class_counts = np.unique(y, return_counts=True)
        self.most_common_label = unique_classes[np.argmax(class_counts)]
        return self
    
    def predict(self, X):
        return np.full((X.shape[0],), self.most_common_label)

# Example usage (Replace X, y with actual data)
# X = np.array([...])
# y = np.array([...])

# pipeline = SimplePipeline([
#     ('scaler', Scaler()),
#     ('encoder', OneHotEncoder()),
#     ('classifier', MostFrequentClassifier())
# ])

# pipeline.fit(X, y)
# predictions = pipeline.predict(X)
